## Dicas de Jupyter Notebook

Este programa que estamos usando no navegador se chama Jupyter Notebook e é um interpretador de Python interativo. Podemos digitar um comando de Python, apertar _Shift + Enter_ e esse comando é executado, tendo seu resultado impresso na tela.

Existem alguns atalhos bastante úteis para trabalhar com Jupyter Notebook:
    0. 'Esc' - Saí do modo edição da célula atual (modo navegação)
    1. 'Enter' - Edita a célula selecionada (modo navegação)
    2. 'Shift + Enter' - Executa a célula selecionada (modo edição)
    3. 'A' - Adiciona uma nova célula acima da célula atual (modo navegação)
    4. 'B' - Adiciona uma nova célula abaixo da célula atual (modo navegação)
    5. 'X' - Recorta a célula selecionada (modo navegação)
    6. 'Z' - Desfaz a deleção de uma célula (modo navegação)
    7. 'Ctrl + Z' - Desfaz as últimas edições (modo edição)
    8. 'Ctrl + Shift + Z' - Refaz o último comando de desfazer (modo edição)
    9. 'H' - Abre um arquivo de ajuda com todos os atalhos (modo navegação)
    10. 'Tab' - Faz sugestões para completar o código (modo edição)
    11. 'Shift + Tab' - Faz sugestões de assinatura de um método ou função (modo edição)
    12. '?função' - Mostra a documentação de uma dada função (modo edição)
    


In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
train['count'].head()

In [ ]:
# nivela as observações para transformar a diferença dos erros em proporção (porcentagem).
# Aproxima mais os erros, p.e., diferença dos valores dos imóveis (independente se é popular ou alto padrão,
# pois o que vai interessar é a diferença (10%, 20%, 50%), pois se tiver muita diferença pode gerar 
# uma oportunidade de negócios

train['count'] = np.log(train['count'])

In [ ]:
train['count'].head()

In [ ]:
import math

In [ ]:
# testando a variação do uso do log
math.log(1000) - math.log(1100)

In [ ]:
# testando a variação do uso do log
math.log(10) - math.log(11)

In [ ]:
#olhando o tamanho das bases
train.shape, test.shape


In [ ]:
# juntando as bases de dados em um dataframe só
# a biblioteca scikit não processa nos modelos colunas tipo texto, apenas numérico. 
# Deve-se criar uma codificação numérica (0 - abacaxi, 1 - laranja, etc)

# junta os dois dataframes. As colunas que não existem no segundo arquivo, ficará como null
df = train.append(test)

In [ ]:
# retornando os tipos dos campos
df.dtypes

In [ ]:
# convertendo campo obj ou texto em data
df['datetime'] = pd.to_datetime(df['datetime'])

In [ ]:
df.dtypes

In [ ]:
df.datetime

In [ ]:
# criando novas colunas com ano, mês, dia, etc

df['year'] = df['datetime'].dt.year

In [ ]:
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['dayofweek'] = df['datetime'].dt.dayofweek
df['hour'] = df['datetime'].dt.hour

In [ ]:
df.head()

In [ ]:
# ordena o dataframe por datetime, a utilização do inplace força a substituição do dataframe

df.sort_values('datetime', inplace=True)

In [ ]:
df.head()

In [ ]:
# imputa a média da temperatura das últimas 4 horas, criando uma nova coluna para armezamento

df['rolling_temp'] = df['temp'].rolling(4, min_periods=1).mean()

In [ ]:
df[['temp', 'rolling_temp']].head()

In [ ]:
# one hot enconding - dummificar variáveis
df.nunique()

In [ ]:
# pegando as 5 primeiras linhas da amostra com os dummies gerados. Foi criado um prefixo ('alias') 
# para facilitar a identificação da sequencia dos dias da semana

pd.get_dummies(df['dayofweek'], prefix='prefixo').head()

In [ ]:
# pega de toda a amostra (" :, "), os registros que possuem valores únicos > 2 e < 8. Neste caso
# utilizando o head() para pegar as 5 primeiras linhas

df.loc[:, (df.nunique() > 2) & (df.nunique() < 8)].head()


In [ ]:
# loop para pegar coluna por coluna, para criar novas colunas para cada ocorrência
# p.e.: na coluna season (4 estações), vai criar 4 colunas dummies, mas como foi usado o comando
# " iloc[:, :-1] ", varre toda a base e não inclui a última coluna e se for tudo zero, a coluna oculta
# será 1 (se não tirar a última coluna pode dá o erro de colinearidade na execução do modelo)
    
for col in df.loc[:, (df.nunique() > 2) & (df.nunique() < 8)].columns:
    df = pd.concat([df, pd.get_dummies(df[col], prefix=col).iloc[:, :-1]], axis=1)
    
    del df[col]



In [ ]:
df.head()

In [ ]:
# cria outro dataframe com os registros não nulos para  coluna ' df['count'] '

train_raw = df[~df['count'].isnull()]

In [ ]:
# cria outro dataframe com os registros nulos para a coluna ' df['count'] '

test = df[df['count'].isnull()]

In [ ]:
train_raw.shape, test.shape

In [ ]:
# Separando em treino e validação

from sklearn.model_selection import train_test_split

In [ ]:
train, valid = train_test_split(train_raw, random_state=42)

In [ ]:
# importa as bibliotecas com os modelos

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR


In [ ]:
# cria um dictionare com os 'alias' dos modelos - chave/valor
# chave é o nome e o modelo em sim é o valor

models = {'RandomForest': RandomForestRegressor(random_state=42),
          'ExtraTrees': ExtraTreesRegressor(random_state=42),
          'GBM': GradientBoostingRegressor(random_state=42),
          'DecisionTree': DecisionTreeRegressor(random_state=42),
          'AdaBoost': AdaBoostRegressor(random_state=42),
          'KNN 1': KNeighborsRegressor(n_neighbors=1),
          'KNN 3': KNeighborsRegressor(n_neighbors=3),
          'KNN 11': KNeighborsRegressor(n_neighbors=11),
          'SVR': SVR(),
          'LinearRegression': LinearRegression()}

In [ ]:
# cria um DF com as colunas que não interessam

removed_cols = ['cont', 'casual', 'registered', 'datetime']

In [ ]:
# cria um novo DF com as colunas que não estão contidas no DF com as colunas removidas
feats = [c for c in df.columns if c not in removed_cols]

In [ ]:
feats

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# cria uma função para executar os modelos de acordo com os dicts acima
# y_name é a variável que queremos prever

def run_model(model, train, valid, feats, y_name):
    model.fit(train[feats], train[y_name]) # comando '.fit' é para treinar o modelo
    preds = model.predict(valid[feats]) # comeando '.predict' é para prever
    return mean_squared_error(valid[y_name], preds)**(1/2) # compara a previsão com o dado real

In [ ]:
# usa a biblioteca para ignorar alguns alertas desnecessários..
import warnings

warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
scores = []
for name, model in models.items():
    score = run_model(model, train, valid, feats, 'count')
    scores.append(score)
    print(name+':', score)

In [ ]:
# plotando gráfico com os resultados

pd.Series(scores, index=models.keys()).sort_values(ascending=False).plot.barh()